In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["Open_API_Key"]=os.getenv("Open_API_Key")

In [16]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [19]:
urls=["https://www.oreilly.com/library/view/learning-python-5th/9781449356862/","https://www.oreilly.com/library/view/python-cookbook-3rd/9781449340373/","https://www.oreilly.com/library/view/python-in-a-nutshell/9781449357333/"]
docs=[]
for url in urls:
    docs.append(WebBaseLoader(url).load())
docss_list =[item for sublist in docs for item in sublist]  # Flatten the list of lists

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
doc_splits = text_splitter.split_documents(docss_list)
vectorstore = FAISS.from_documents(documents=doc_splits, embedding=HuggingFaceBgeEmbeddings())
retrieverpython= vectorstore.as_retriever()

from langchain.tools.retriever import create_retriever_tool

retriever_tool_python = create_retriever_tool(
    retrieverpython,
    "retriever_vector_python",
    "A tool to retrieve Python-related information from a vector store.",
)

C:\Users\Kavyasree P\AppData\Local\Temp\ipykernel_26632\1152939508.py:9: LangChainDeprecationWarning: Default values for HuggingFaceBgeEmbeddings.model_name were deprecated in LangChain 0.2.5 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceBgeEmbeddings constructor instead.
  vectorstore = FAISS.from_documents(documents=doc_splits, embedding=HuggingFaceBgeEmbeddings())


In [20]:
lang_chain_urls = [
    "https://python.langchain.com/docs/get_started/introduction.html",
    "https://python.langchain.com/docs/get_started/quickstart.html",
    "https://python.langchain.com/docs/get_started/faq.html",
]
docs_langchain = []
for url in lang_chain_urls:
    docs_langchain.append(WebBaseLoader(url).load())
docss_list = [item for sublist in docs_langchain for item in sublist]  # Flatten the list of lists

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
doc_splits = text_splitter.split_documents(docss_list)
vectorstore = FAISS.from_documents(documents=doc_splits, embedding=HuggingFaceBgeEmbeddings())
retrieverlangchain = vectorstore.as_retriever()

retrieval_tool_langchain = create_retriever_tool(
    retrieverlangchain,
    "retriever_vector_langchain",
    "A tool to retrieve LangChain-related information from a vector store.",
)

C:\Users\Kavyasree P\AppData\Local\Temp\ipykernel_26632\1400811609.py:13: LangChainDeprecationWarning: Default values for HuggingFaceBgeEmbeddings.model_name were deprecated in LangChain 0.2.5 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceBgeEmbeddings constructor instead.
  vectorstore = FAISS.from_documents(documents=doc_splits, embedding=HuggingFaceBgeEmbeddings())


In [ ]:
tools= [retriever_tool_python,retrieval_tool_langchain]

In [ ]:
%pip install langgraph

from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_message


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_message]



In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    model="groq-llama-3.1-70b-chat")
llm.invoke("What is LangChain?")

In [ ]:
def agent(state):
    print("___CALLING AGENT ___")
    message=state["message"]
    model= ChatGroq(
        model="groq-llama-3.1-70b-chat",
    )   
    model=model.bind_tools(tools)
    response=model.invoke(message)
    return {"message": [response]}

In [ ]:
from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict

from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

from pydantic import BaseModel, Field

In [ ]:
def grade_documents(state) -> Literal["generate","rewrite"]:
    '''Determines whether the retrieved documents are relevant to the question.

Args :
state (messages): The current state

Returns :
str: A decision for whether the documents are relevant or not'''
    print("___CHECKING RELEVANCE ___")

    # Data model
    class grade(BaseModel):
        """Binary score for relevance check. """
        binary_score: str = Field(description="Relevance score 'yes' or 'no'")

    # LLM
    model = ChatGroq(model="qwen-qwq-32b")

    # LLM with tool and validation
    llm_with_tool = model.with_structured_output(grade)

    prompt = PromptTemplate(
    template="""You are a grader assessing relevance of a retrieved document to a user. Here is the retrieved documeht: \n\n {context} \n\n
    Here is the user question: {question} \n
    If the document contains keyword(s) or semantic meaning related to the user questior
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant"""
    input_variables=["context", "question"],
    )
    # Chain
    chain = prompt | llm_with_tool

    messages = state["messages"]
    last_message = messages[-1]

    question = messages[0].content
    docs = last_message.content

    scored_result = chain.invoke({"question": question, "context": docs})

In [ ]:
graph.invoke("What is LangChain?")